<a href="https://colab.research.google.com/github/para662/dataviz2025/blob/main/midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import pandas as pd

# ให้ pandas แสดงตารางเต็ม ไม่ห่อบรรทัด
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 200)

# ให้แสดงคอลัมน์เต็ม ไม่ตัดเป็น ...
pd.set_option('display.max_colwidth', None)


In [51]:
import pandas as pd

crime = pd.read_csv("/content/drive/MyDrive/mid data/crime_scene_report.csv")

case = crime[
    (crime["date"] == 20180115) &
    (crime["type"] == "murder") &
    (crime["city"] == "Pandas City")
]

print(case)


          date    type                                                                                                                                                                                description         city
1227  20180115  murder  Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".  Pandas City


In [49]:
import pandas as pd

person = pd.read_csv("/content/drive/MyDrive/person.csv")

w1 = (person[person["address_street_name"] == "Northwestern Dr"]
      .sort_values("address_number", ascending=False)
      .head(1))

w2 = person[
    (person["name"].str.contains("Annabel")) &
    (person["address_street_name"] == "Franklin Ave")
]

print("พยาน 1:\n", w1[["id","name","address_number","address_street_name"]], "\n")
print("พยาน 2:\n", w2[["id","name","address_number","address_street_name"]])


พยาน 1:
         id            name  address_number address_street_name
499  14887  Morty Schapiro            4919     Northwestern Dr 

พยาน 2:
         id            name  address_number address_street_name
665  16371  Annabel Miller             103        Franklin Ave


In [52]:
import pandas as pd

interview = pd.read_csv("/content/drive/MyDrive/interview.csv")

# ดึงคำให้การของ id = 14887 และ 16371
iv_targets = interview[interview["person_id"].isin([14887, 16371])]

print(iv_targets)


      person_id                                                                                                                                                                                                                       transcript
4988      14887  I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
4989      16371                                                                                                            I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


In [57]:
#หาคนไปยิมวันที่ 20180109
CHECKIN_PATH = "/content/drive/MyDrive/get_fit_now_check_in.csv"
#หาสมาชิกระดับ gold
MEMBER_PATH  = "/content/drive/MyDrive/get_fit_now_member.csv"
#หาrisenรถ
PERSON_PATH  = "/content/drive/MyDrive/person.csv"
#หารถ "H42W"
DL_PATH      = "/content/drive/MyDrive/drivers_license.csv"

checkin = pd.read_csv(CHECKIN_PATH)
member  = pd.read_csv(MEMBER_PATH)
person  = pd.read_csv(PERSON_PATH)
dl      = pd.read_csv(DL_PATH)

#กรองคนที่ไปยิมวันที่ 20180109
checkin["check_in_date"] = checkin["check_in_date"].astype(str)
ci_0109 = checkin[checkin["check_in_date"] == "20180109"]

#ต่อกับรายชื่อสมาชิก จะได้ชื่อและสถานะสมาชิก
ci_0109 = ci_0109.merge(member, left_on="membership_id", right_on="id", how="left")

# กรองตามใบ้จากพยาน: รหัสขึ้นต้น 48Z และเป็น gold
sus = ci_0109[
    ci_0109["membership_id"].astype(str).str.startswith("48Z") &
    (ci_0109["membership_status"].str.lower() == "gold")
]

#  หา license_id ของคนแต่ละคน
sus = sus.merge(person, left_on="person_id", right_on="id", how="left", suffixes=("", "_person"))

# ต่อกับ drivers_license เพื่อดูทะเบียนรถ
sus = sus.merge(dl, left_on="license_id", right_on="id", how="left", suffixes=("", "_dl"))

# กรองทะเบียนที่มี H42W
killer = sus[sus["plate_number"].str.contains("H42W", case=False, na=False)]

# แสดงผลที่ต้องการดู
cols = ["name", "membership_id", "membership_status", "check_in_time", "check_out_time",
        "address_number", "address_street_name", "plate_number", "car_make", "car_model"]
print(killer[cols])



            name membership_id membership_status  check_in_time  check_out_time  address_number    address_street_name plate_number   car_make car_model
1  Jeremy Bowers         48Z55              gold           1530            1700             530  Washington Pl, Apt 3A       0H42W2  Chevrolet  Spark LS


In [60]:
# Find Jeremy Bowers in the person DataFrame
jeremy_bowers = person[person['name'] == 'Jeremy Bowers']
# Extract the person ID
jeremy_id = jeremy_bowers['id'].iloc[0]

print("Jeremy Bowers' person ID:", jeremy_id)

Jeremy Bowers' person ID: 67318


In [61]:
# Find the interview for the person with ID 67318
suspect_interview = interview[interview['person_id'] == 67318]

display("Interview transcript for person ID 67318:")
display(suspect_interview['transcript'].iloc[0])

'Interview transcript for person ID 67318:'

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

In [63]:
# Load the facebook event check-in data
# Assuming the file path is correct
try:
    fb_df = pd.read_csv('/content/drive/MyDrive/facebook_event_checkin.csv')
    display("Successfully loaded facebook_event_checkin.csv")

    # Filter for only "SQL Symphony Concert" events
    sql_concert_checkins = fb_df[fb_df['event_name'] == 'SQL Symphony Concert']

    display("Check-ins for SQL Symphony Concert:")
    display(sql_concert_checkins)

except FileNotFoundError:
    display("Error: facebook_event_checkin.csv not found. Please check the file path.")

'Successfully loaded facebook_event_checkin.csv'

'Check-ins for SQL Symphony Concert:'

,person_id,event_id,event_name,date
11,97207,1143,SQL Symphony Concert,20170322
12,97207,1143,SQL Symphony Concert,20180226
109,92433,1143,SQL Symphony Concert,20170821
110,92433,1143,SQL Symphony Concert,20180327
296,88952,1143,SQL Symphony Concert,20170918
...,...,...,...,...
20003,24556,1143,SQL Symphony Concert,20171224
20006,99716,1143,SQL Symphony Concert,20171206
20007,99716,1143,SQL Symphony Concert,20171212
20008,99716,1143,SQL Symphony Concert,20171229


In [64]:
# Filter for SQL Symphony Concert check-ins in December 2017
sql_concert_dec2017 = sql_concert_checkins[
    (sql_concert_checkins['date'] >= 20171201) &
    (sql_concert_checkins['date'] <= 20171231)
]

# Count the number of check-ins for each person at the SQL Symphony Concert in December 2017
concert_counts = sql_concert_dec2017['person_id'].value_counts().reset_index()
concert_counts.columns = ['person_id', 'times']

# Filter for people who attended 3 times
attended_3_times_dec2017 = concert_counts[concert_counts['times'] == 3]
display("People who attended SQL Symphony Concert 3 times in December 2017:")
display(attended_3_times_dec2017)

# Now we have the person ID(s) of the potential mastermind(s) based on concert attendance.
# Let's cross-reference this with other clues from Jeremy's interview (name Annabel, red hair, height 5'5"-5'7", drives a Tesla Model S, and has a lot of money) to confirm the mastermind.

'People who attended SQL Symphony Concert 3 times in December 2017:'

,person_id,times
0,24556,3
1,99716,3


In [73]:

PERSON_PATH = "/content/drive/MyDrive/person.csv"

person = pd.read_csv(PERSON_PATH)

# กรองหาเฉพาะ id ที่สนใจ
targets = person[person["id"].isin([24556, 99716])]

# จัดเรียงตามลำดับ id ที่เรากำหนดเอง
targets = targets.set_index("id").loc[[24556, 99716]].reset_index()

print(targets)



      id              name  license_id  address_number address_street_name        ssn
0  24556       Bryan Pardo      101191             703          Machine Ln  816663882
1  99716  Miranda Priestly      202298            1883          Golden Ave  987756388


In [71]:
import pandas as pd

INCOME_PATH = "/content/drive/MyDrive/income.csv"

income = pd.read_csv(INCOME_PATH)

# กรองหา ssn เป้าหมาย
targets = income[income["ssn"].isin([816663882, 987756388])]

print(targets[["ssn", "annual_income"]])


            ssn  annual_income
7422  987756388         310000



# คำตอบ  
ฆาตกร       : Jeremy Bowers  
ผู้บงการ     : Miranda Priestly (รายได้ ~ 310000)  
เพราะ อีกคนไม่มีเงิน income เเต่คำให้การบอกรวย
